In [1]:
import numpy as np
image_shape = [227,227,3]
np.random.seed(1000)
from keras.models import Sequential
import keras
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('another/training_set',
                                                 target_size = (227, 227),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

# Preprocessing the Test set
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('another/test_set',
                                            target_size = (227, 227),
                                            batch_size = 32,
                                            class_mode = 'binary')
model = Sequential()

model.add(Conv2D(filters=96,input_shape=image_shape,kernel_size=(11,11),strides=(4,4),padding='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='valid'))

model.add(Conv2D(filters=256,kernel_size=(5,5),strides=(1,1),padding='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='valid'))

model.add(Conv2D(filters=384,kernel_size=(3,3),strides=(1,1),padding='valid'))
model.add(Activation('relu'))

model.add(Conv2D(filters=384,kernel_size=(3,3),strides=(1,1),padding='valid'))
model.add(Activation('relu'))

model.add(Conv2D(filters=256,kernel_size=(3,3),strides=(1,1),padding='valid'))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='valid'))

model.add(Flatten())

model.add(Dense(4096))
model.add(Activation('relu'))

model.add(Dense(4096))
model.add(Activation('relu'))

#model.add(Dropout(0.4))
model.add(Dense(1000))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary()

opt = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss=keras.losses.binary_crossentropy,optimizer=opt,metrics=['accuracy'])

from keras import callbacks 
earlystopping = callbacks.EarlyStopping(monitor ="val_loss",  
                                        mode ="min", patience = 5,  
                                        restore_best_weights = True)
hist = model.fit(x = training_set, validation_data= test_set, epochs=10,callbacks = [earlystopping])

Found 195 images belonging to 2 classes.
Found 58 images belonging to 2 classes.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 55, 55, 96)        34944     
_________________________________________________________________
activation (Activation)      (None, 55, 55, 96)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 23, 23, 256)       614656    
_________________________________________________________________
activation_1 (Activation)    (None, 23, 23, 256)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 11, 11, 256)       0         
_________________________________________

In [2]:
(loss,accuracy) = model.evaluate(test_set,batch_size=64)
print("Accuracy")
print(accuracy)

2/2 [==============================] - 1s 263ms/step - loss: 0.0659 - accuracy: 0.9828
Accuracy
0.982758641242981


In [5]:
import numpy as np 
from keras.preprocessing import image 
test_image = image.load_img('try/single_prediction/S3.jpg', target_size=(227, 227)) 
test_image = image.img_to_array(test_image) 
print(test_image.shape) 
test_image = np.expand_dims(test_image, axis = 0) 
result = model.predict(test_image) 
training_set.class_indices 
if result[0][0] == 1: 
    print("Tumor") 
else: 
    print("No Tumor")

(227, 227, 3)
Tumor
